# Description

In this exercise, we continue to work with the small "numbertheory" library used in prior exercises.  However, a couple new functions have been added for this exercise.  These lend themselves to testing with fixtures. Let us look at the help on these new functions:

```
likely_prime(n: int, k: int = 20) -> bool
    Test if a number is VERY LIKELY to be prime
    
    Implementation uses the Miller-Rabin Primality Test
    
    n: Natural Number to check
    k: number of rounds (default 20)
    
    Default 20 rounds wrong less than one in trillion 
    40 rounds less likely than cosmic ray interfering
    
    return:  Boolean answer
```
---
```
count_primes_in_file(filename: str) -> Ratio
    Count the fraction of primes within a text file
    
    filename: file of Natural Numbers as strings, once per line
```
---
```
random_uint64_to_file(filename: str = None, count: int = 10000, _inc=[1])
    Create a file of Natural Numbers up to 2^64-1
    
    filename: the file where numbers will be saved
    count: number of numbers to write, one per line (default 100)
    
    Uses /dev/urandom as source of randomness rather than Python
    pseudo-random Mersenne Twister algorithm
    
    return: filename (generated if not provided)
```

You should construct a number of additional tests for the behavior of these functions, many of them are best expressed using fixtures.  In particular, those that involve creating files should cleanup after themselves.  Some of these tests may benefit from being parameterized into multiple tests.  The setup section gives skeletons of tests you should write, but not implementations.

Note: An **exact** prime counting function would produce the values listed (no known closed form expression is exact). 

|  x       |  π(x)
|---------:|---------
|       10 | 4
|      100 | 25
|     1000 | 168
|   10,000 | 1,229
|  100,000 | 9,592


# Setup

In [18]:
from numbertheory.utilities import *
import pytest

def test_primes_are_likely_primes():
    # Create a file of many primes
    # Test that all primes in that file pass Miller Rabin
    # Delete file when done working with it
    assert False, "Some primes do not pass primality test"
    
def test_primality_randoms():
    # Create many files of random numbers using
    # random_uint64_to_file()
    # Statistically, 10,000 random 64-bit ints will
    # almost always have between 170 and and 295 primes
    assert False, "Random numbers have unexpected prime frequency"
    
def test_true_prime_count():
    # Create files of the initial 10^N integer and
    # verify that likely_prime() applied to all
    # numbers produces exact answer for prime counting
    assert False, "Identified primes in file have wrong number"


# Solution

In [28]:
import pytest
from pathlib import Path
#from utilities import *


@pytest.fixture(params=list(range(10)))
def randoms_uint64(request):
    # Do not actually use parameter, just want to test multiple
    count = 10_000
    fname = random_uint64_to_file(count=count)
    yield fname, count
    Path(fname).unlink()


@pytest.fixture(scope="session", params=list(range(1, 6)))
def number_file(request):
    n = 10**request.param
    fname = f'numbers-{n}.txt'
    with open(fname, 'w') as fh:
        for i in range(1, n+1):
            print(i, file=fh)
    yield fname, request.param
    Path(fname).unlink()


@pytest.fixture(scope="session")
def primes_5000():
    fname = 'primes-5000.txt'
    with open(fname, 'w') as fh:
        for i in get_init_primes(5000):
            print(i, file=fh)
    yield fname, 5000
    Path(fname).unlink()

    
@pytest.fixture
def exact_prime_count():
    return {1:4, 2:25, 3:168, 4:1229, 5:9592, 6:78_498}


def test_true_prime_count(number_file, exact_prime_count):
    fname, nlog = number_file
    pnum = count_primes_in_file(fname)
    assert exact_prime_count[nlog] == pnum.numerator


def test_primes_are_likely_primes(primes_5000):
    fname, count = primes_5000
    pnum = count_primes_in_file(fname)
    assert count == pnum.numerator


def test_primality_randoms(randoms_uint64):
    fname, n = randoms_uint64
    if n != 10_000:
        assert False, f"Distribution not checked for size {n}"
    pnum = count_primes_in_file(fname)
    assert 170 < pnum.numerator < 295

# Test Cases

In [32]:
def test_enough_tests():
    tests = {name for name in globals() if name.startswith('test_')}
    for name in ['test_true_prime_count', 
                 'test_primes_are_likely_primes',
                 'test_primality_randoms']:
        assert name in tests
    
test_enough_tests()